In [9]:
# 📦 Importa librerías
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
import os

# 📁 Ruta del CSV
csv_path = "/home/user/proyecto-nitro/notebooks/data/enhanced_predictions.csv"

# ✅ Carga el DataFrame
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"📄 CSV cargado con {df.shape[0]} filas y {df.shape[1]} columnas.")
    mlflow.log_artifact(csv_path)
else:
    print(f"⚠️ Archivo no encontrado: {csv_path}")
    df = None

# 📁 Configura MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Nitro_Sensor_Anomalies")

# 🧼 Cierra ejecución previa si está activa
if mlflow.active_run() is not None:
    mlflow.end_run()

# 🚀 Inicia el tracking
with mlflow.start_run(run_name="Nitro_Ensemble_Run"):
    mlflow.log_params({
        "model_type": "IsolationForest + RandomForest",
        "contamination": 0.2,
        "threshold_percentile": 80
    })

    if df is not None:
        # ✅ Verifica si hay columna 'anomaly'
        if 'anomaly' in df.columns:
            try:
                # 🔍 Features y target
                feature_columns = [col for col in df.columns if col != 'anomaly']
                X = df[feature_columns]
                y = df['anomaly']

                # 🧠 Entrena modelos
                prev_model = RandomForestClassifier()
                prev_model.fit(X, y)

                iso_forest = IsolationForest(contamination=0.2)
                iso_forest.fit(X)

                # 🔮 Predicción
                final_pred = prev_model.predict(X)

                # 📊 Métricas
                precision = precision_score(y, final_pred)
                recall = recall_score(y, final_pred)
                mlflow.log_metrics({
                    "precision": precision,
                    "recall": recall
                })
                print(f"📈 Métricas loggeadas: Precision={precision:.3f}, Recall={recall:.3f}")

                # 🧠 Loguea modelos
                mlflow.sklearn.log_model(prev_model, "random_forest_model")
                mlflow.sklearn.log_model(iso_forest, "isolation_forest")

            except Exception as e:
                print(f"❌ Error durante el entrenamiento o loggeo: {e}")
        else:
            print("⚠️ El DataFrame no tiene la columna 'anomaly'.")
    else:
        print("⚠️ El DataFrame no fue cargado correctamente.")

print("✅ MLflow tracking completado")

📄 CSV cargado con 5 filas y 7 columnas.
🏃 View run bald-worm-328 at: http://localhost:5000/#/experiments/1/runs/68ab6c6bfc5941a08507f902d198911c
🧪 View experiment at: http://localhost:5000/#/experiments/1
⚠️ El DataFrame no tiene la columna 'anomaly'.
🏃 View run Nitro_Ensemble_Run at: http://localhost:5000/#/experiments/1/runs/af255b73f2bf4aac9cddeff5146710ad
🧪 View experiment at: http://localhost:5000/#/experiments/1
✅ MLflow tracking completado
